In [0]:
from google.colab import drive
drive.mount('/content/drive')
import keras
import numpy as np
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Activation, Dense,Conv2D,MaxPooling2D
from keras.layers import Dropout, Flatten
from keras.utils import np_utils
from keras import optimizers
from keras.layers.normalization import BatchNormalization
import random


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


 **if you're upload your zipped dataset**

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('drive/My Drive/dataset_filepath.zip', 'r')
zip_ref.extractall('drive/My Drive/Colab Notebooks')
zip_ref.close()

***import dataset as numpy array*** 

In [0]:
x_train = np.load("drive/My Drive/Colab Notebooks/dataset.npy")
y = np.load("drive/My Drive/Colab Notebooks/label.npy")
x_train = x_train.reshape(-1,32, 32, 1)
classes = np.unique(y)
nClasses =len(classes)
y_train = keras.utils.to_categorical(y,num_classes=nClasses)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)
print(y[0])
print(len(x_train))


Total number of outputs :  42
Output classes :  [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41.]
37.0
752215


**Model Creating**

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=[32,32,1]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))

model.add(Flatten())

model.add(Dense(288, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(144, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(72, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
batch_normalization_16 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 32)        9248      
_________________________________________________________________
batch_normalization_17 (Batc (None, 15, 15, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 64)          18496     
__________

**This project we using multi-label classification and we use k-fold cross-validation for getting good accuracy 

In [0]:
c = list(zip(x_train,y_train))
random.shuffle(c)
x_train,y_train = zip(*c)
del c
k=10
length = len(x_train)
kfoldLen=int(length/k)
score = []

for x in range(k):
  testX=[]
  testY=[]
  trainX=[]
  trainY=[]

  split_init = kfoldLen*x
  split_end = split_init+kfoldLen
  testX = x_train[split_init:split_end]
  testY = y_train[split_init:split_end]
  if x!=0:
    trainX = x_train[0:split_init]
    trainY = y_train[0:split_init]
    if x!=k-1:
      trainX+=x_train[split_end:length-1]
      trainY+=y_train[split_end:length-1]
  else:
    trainX=x_train[split_end:length-1]
    trainY=y_train[split_end:length-1]
   
  trainX = np.asarray(trainX)
  trainY = np.asarray(trainY)
  testX = np.asarray(testX)
  testY = np.asarray(testY)
  
  opt = keras.optimizers.Adam(lr=0.001)
  print("------Fold: "+str(x+1)+"------")
  model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
  history = model.fit(trainX,trainY,epochs=10,shuffle=False,verbose=1,batch_size=256)
  foldScore = model.evaluate(testX,testY)
  score.append(foldScore[1]*100)
  print("-------------Test Score ----------")
  print("Accuracy: "+str(foldScore[1]*100))  
  print("Loss: "+str(foldScore[0]*100)) 
  
print("%.2f%% (+/- %.2f%%)" % (np.mean(score), np.std(score)))

model.save("drive/My Drive/Colab Notebooks/hndwrte_recog_model.h5")

W0618 12:13:23.867762 139758212724608 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0618 12:13:24.008913 139758212724608 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


------Fold: 1------
Epoch 1/10
676993/676993 [==============================] - 91s 134us/step - loss: 0.4351 - acc: 0.8678
Epoch 2/10
676993/676993 [==============================] - 85s 126us/step - loss: 0.2690 - acc: 0.9111
Epoch 3/10
676993/676993 [==============================] - 85s 125us/step - loss: 0.2437 - acc: 0.9185
Epoch 4/10
676993/676993 [==============================] - 85s 126us/step - loss: 0.2296 - acc: 0.9223
Epoch 5/10
676993/676993 [==============================] - 86s 126us/step - loss: 0.2181 - acc: 0.9257
Epoch 6/10
676993/676993 [==============================] - 85s 126us/step - loss: 0.2113 - acc: 0.9275
Epoch 7/10
676993/676993 [==============================] - 85s 126us/step - loss: 0.2042 - acc: 0.9297
Epoch 8/10
676993/676993 [==============================] - 85s 126us/step - loss: 0.1989 - acc: 0.9310
Epoch 9/10
676993/676993 [==============================] - 85s 126us/step - loss: 0.1946 - acc: 0.9322
Epoch 10/10
75221/75221 [===================